贝叶斯分类：对于给定数据，判断其属于哪个类别。通过计算所有p（Ci|x,y)，概率最大对应类别就是分类结果。计算过程中用到贝叶斯规则，将计算转化为P（x,y|Ci),p(Ci)计算。
之所以称为朴素：即假设所有属性概率上条件独立

In [16]:
import numpy as np

def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1] #1 is abuse , 0 not
    return postingList, classVec

def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
        
    return list(vocabSet)
 
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print ("the word : %s is not in my Vocabulary!" % word)
    return returnVec

a, _= loadDataSet()
b = createVocabList(a)

def trainNaiveBayes(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num = np.zeros(numWords)
    p1Num = np.zeros(numWords)
    p0Denom = 0.0
    p1Denom = 0.0
    
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
      
    p1Vect = np.log(p1Num / p1Denom)
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive

def classifyNB(vect2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vect2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vect2Classify * p0Vec) + np.log(1 - pClass1)
    
    if p1 > p0:
        return 1
    else:
        return 0
    
def testNB():
    listPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listPosts)
    trainMat = []
    for postinDoc in listPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNaiveBayes(np.array(trainMat), np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry)
    print('classified as %d '% classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ['tupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry)
    print('classified as %d '% classifyNB(thisDoc, p0V, p1V, pAb))
    
testNB()

['love', 'my', 'dalmation']
classified as 0 
the word : tupid is not in my Vocabulary!
['tupid', 'garbage']
classified as 0 


C:\Users\wupin\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log
C:\Users\wupin\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in log
C:\Users\wupin\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in multiply
C:\Users\wupin\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in multiply
